<a href="https://colab.research.google.com/github/run-llama/llama_parse/blob/main/examples/excel/o1_excel_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced RAG with LlamaParse + OpenAI O1 on Excel

This notebook provides a reference to compare advanced RAG capabilities using LlamaParse with `o1-preview`, `o1-mini` and `gpt4o-mini` on a sample Excel document.

We will use `2Q 2024 Group databook - xls` file from [bp.com](https://www.bp.com/en/global/corporate/investors/results-reporting-and-presentations/financial-disclosure-framework/archive.html) for the demonstration.

When interacting with our enterprise customers, we've identified two prominent types of queries. Let's check how they perform with the o1 models:

1. Queries requesting exact values.
2. Queries using the greater than/less than (>/ <) operators.

#### Installation

In [ ]:
# !pip install llama-index
# !pip install llama-parse

## Import

In [ ]:
import nest_asyncio

from llama_index.llms.openai import OpenAI
from llama_index.core import VectorStoreIndex
from IPython.display import Markdown, display

from llama_parse import LlamaParse

from llama_index.core.node_parser import MarkdownElementNodeParser

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "sk-..."

#### Some OpenAI and LlamaParse details

In [ ]:
# llama-parse is async-first, running the async code in a notebook requires the use of nest_asyncio
nest_asyncio.apply()

#### Setup LLM

In [ ]:
llm_o1 = OpenAI(model="o1-mini")
llm_gpt4o_mini = OpenAI(model="gpt-4o-mini")
llm_o1_preview = OpenAI(model="o1-preview")

## Using brand new `LlamaParse` PDF reader for PDF Parsing


We will use `MarkdownElementNodeParser` for parsing the `LlamaParse` output Markdown results and building recursive retriever query engine for generation.

#### LlamaParse

In [ ]:
parser = LlamaParse(
    api_key="llx-...",
    result_type="markdown",
)

documents = parser.load_data("./data/BP_Excel.xlsx")

Started parsing the file under job_id 9481484f-4414-4f20-aad8-892dc57649a1


In [ ]:
len(documents)

44

In [ ]:
print(documents[3].get_content())

# Summary

|Financial and Operating Information 2020 - 2024                          |           |        |        |        |        |        |        |        |        |        |        |         |        |        |        |        |        |        |        |        |        |        |        |      |      |           |
|-------------------------------------------------------------------------|-----------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|------|------|-----------|
|Group information                                                        |           |        |        |        |        |        |        |        |        |        |        |         |        |        |        |        |        |        |        |        |        |        |        |      |      |           |
|                                                 

#### MarkdownElementNodeParser

This will generate a summary for each node; if a table is present, it will also create a summary for the table.

In [ ]:
node_parser = MarkdownElementNodeParser(llm=llm_gpt4o_mini, num_workers=4)

### Parse the documents

In [ ]:
nodes = node_parser.get_nodes_from_documents(documents[:10])

1it [00:00, 18558.87it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
1it [00:00, 8630.26it/s]
1it [00:00, 18157.16it/s]
1it [00:00, 8355.19it/s]
1it [00:00, 5053.38it/s]
1it [00:00, 6955.73it/s]
1it [00:00, 7626.01it/s]
1it [00:00, 4832.15it/s]


In [ ]:
base_nodes, objects = node_parser.get_nodes_and_objects(nodes)

In [ ]:
len(nodes), len(base_nodes), len(objects)

(27, 11, 8)

In [ ]:
print(objects[3].get_content())

This table presents the financial and operating information of a group from 2020 to 2024, detailing the condensed group statement of comprehensive income, including profit or loss for the period, other comprehensive income, and total comprehensive income attributable to shareholders and non-controlling interests.,
with the following table title:
Financial and Operating Information 2020 - 2024,
with the following columns:
- Group information: None
- Condensed group statement of comprehensive income: None
- Profit (loss) for the period: None
- Other comprehensive income: None
- Total comprehensive income: None
- Attributable to bp shareholders: None
- Attributable to non-controlling interests: None



In [ ]:
len(base_nodes), len(objects)

(11, 8)

#### Build Recursive Retrieval Index

In [ ]:
# dump both indexed tables and page text into the vector index
recursive_index = VectorStoreIndex(nodes=base_nodes + objects, llm=llm_gpt4o_mini)

recursive_query_engine_o1 = recursive_index.as_query_engine(
    similarity_top_k=5, llm=llm_o1
)

recursive_query_engine_o1_preview = recursive_index.as_query_engine(
    similarity_top_k=5, llm=llm_o1_preview
)

recursive_query_engine_gpt4o_mini = recursive_index.as_query_engine(
    similarity_top_k=5, llm=llm_gpt4o_mini
)

# Testing queries

### <font color="#2244FF">Query 1</font>

Expected Answer:

$105,944 Million

In [ ]:
query = "What is the Sales and other operating revenues in 2020?"

response_recursive_o1 = recursive_query_engine_o1.query(query)
response_recursive_o1_preview = recursive_query_engine_o1_preview.query(query)
response_recursive_gpt4o_mini = recursive_query_engine_gpt4o_mini.query(query)

In [ ]:
print("----------------------RESPONSE WITH O1 MINI----------------------")
display(Markdown(f"{response_recursive_o1}"))

print("----------------------RESPONSE WITH O1 PREVIEW----------------------")
display(Markdown(f"{response_recursive_o1_preview}"))

print("----------------------RESPONSE WITH GPT4O-MINI----------------------")
display(Markdown(f"{response_recursive_gpt4o_mini}"))

----------------------RESPONSE WITH O1 MINI----------------------


In 2020, the Sales and Other Operating Revenues amounted to **$105,944 million**.

----------------------RESPONSE WITH O1 PREVIEW----------------------


In 2020, the Sales and other operating revenues were $105,944 million.

----------------------RESPONSE WITH GPT4O-MINI----------------------


The Sales and other operating revenues in 2020 amount to 105,944 million dollars.

### <font color="#2244FF">Query 2</font>

Expected Answer:

2021, 2022, 2023

In [ ]:
query = "In which years the Sales and other operating revenues is greater than $1,50,000 million?"

response_recursive_o1 = recursive_query_engine_o1.query(query)
response_recursive_o1_preview = recursive_query_engine_o1_preview.query(query)
response_recursive_gpt4o_mini = recursive_query_engine_gpt4o_mini.query(query)

In [ ]:
print("----------------------RESPONSE WITH O1 MINI----------------------")
display(Markdown(f"{response_recursive_o1}"))

print("----------------------RESPONSE WITH O1 PREVIEW----------------------")
display(Markdown(f"{response_recursive_o1_preview}"))

print("----------------------RESPONSE WITH GPT4O-MINI----------------------")
display(Markdown(f"{response_recursive_gpt4o_mini}"))

----------------------RESPONSE WITH O1 MINI----------------------


The years in which the Sales and other operating revenues exceeded $150,000 million are 2021, 2022, and 2023.

----------------------RESPONSE WITH O1 PREVIEW----------------------


The Sales and other operating revenues were greater than $150,000 million in the years 2021, 2022, and 2023.

----------------------RESPONSE WITH GPT4O-MINI----------------------


The Sales and other operating revenues exceed $150,000 million in the years 2022 and 2023.

### <font color="#2244FF">Query 3</font>

In [ ]:
query = "Which quarters and years has Total revenues and other income greater than $35K million?"

response_recursive_o1 = recursive_query_engine_o1.query(query)
response_recursive_o1_preview = recursive_query_engine_o1_preview.query(query)
response_recursive_gpt4o_mini = recursive_query_engine_gpt4o_mini.query(query)

In [ ]:
print("----------------------RESPONSE WITH O1 MINI----------------------")
display(Markdown(f"{response_recursive_o1}"))

print("----------------------RESPONSE WITH O1 PREVIEW----------------------")
display(Markdown(f"{response_recursive_o1_preview}"))

print("----------------------RESPONSE WITH GPT4O-MINI----------------------")
display(Markdown(f"{response_recursive_gpt4o_mini}"))

----------------------RESPONSE WITH O1 MINI----------------------


Between 2021 and 2023, all four quarters each year exceeded $35,000 million in total revenues and other income. Additionally, both the first and second quarters of 2024 also surpassed this threshold.

----------------------RESPONSE WITH O1 PREVIEW----------------------


Total revenues and other income exceeded $35,000 million in the following periods:

**Quarters:**
- Q1 2021
- Q2 2021
- Q3 2021
- Q4 2021
- Q1 2022
- Q2 2022
- Q3 2022
- Q4 2022
- Q1 2023
- Q2 2023
- Q3 2023
- Q4 2023
- Q1 2024
- Q2 2024

**Years:**
- 2020
- 2021
- 2022
- 2023

----------------------RESPONSE WITH GPT4O-MINI----------------------


The quarters and years where Total revenues and other income exceeded $35,000 million are:

- Q1 2021: $36,492 million
- Q2 2021: $37,598 million
- Q3 2021: $37,867 million
- Q4 2021: $52,238 million
- Q1 2022: $51,220 million
- Q2 2022: $69,506 million
- Q3 2022: $57,809 million
- Q4 2022: $70,356 million
- Q1 2023: $56,951 million
- Q2 2023: $49,479 million
- Q3 2023: $54,016 million
- Q4 2023: $52,586 million
- Q1 2024: $49,961 million
- Q2 2024: $48,250 million

### <font color="#2244FF">Query 4</font>

Expected Answer:

Q1

In [ ]:
query = (
    "Which quarters in 2023 is Total comprehensive income greater than $9000 million?"
)

response_recursive_o1 = recursive_query_engine_o1.query(query)
response_recursive_o1_preview = recursive_query_engine_o1_preview.query(query)
response_recursive_gpt4o_mini = recursive_query_engine_gpt4o_mini.query(query)

In [ ]:
print("----------------------RESPONSE WITH O1 MINI----------------------")
display(Markdown(f"{response_recursive_o1}"))

print("----------------------RESPONSE WITH O1 PREVIEW----------------------")
display(Markdown(f"{response_recursive_o1_preview}"))

print("----------------------RESPONSE WITH GPT4O-MINI----------------------")
display(Markdown(f"{response_recursive_gpt4o_mini}"))

----------------------RESPONSE WITH O1 MINI----------------------


In 2023, the Total Comprehensive Income exceeded $9,000 million in the first quarter.

----------------------RESPONSE WITH O1 PREVIEW----------------------


In the second quarter (Q2) of 2023, the total comprehensive income exceeded $9,000 million.

----------------------RESPONSE WITH GPT4O-MINI----------------------


In 2023, the quarters where Total comprehensive income is greater than $9000 million are Q2 and Q4.

### <font color="#2244FF">Query 5</font>

Expected Answer:

$392 million

In [ ]:
query = "what is the replacement cost profit (loss) in oil production & operations URCP post taxation in Q1 of 2020?"

response_recursive_o1 = recursive_query_engine_o1.query(query)
response_recursive_o1_preview = recursive_query_engine_o1_preview.query(query)
response_recursive_gpt4o_mini = recursive_query_engine_gpt4o_mini.query(query)

In [ ]:
print("----------------------RESPONSE WITH O1 MINI----------------------")
display(Markdown(f"{response_recursive_o1}"))

print("----------------------RESPONSE WITH O1 PREVIEW----------------------")
display(Markdown(f"{response_recursive_o1_preview}"))

print("----------------------RESPONSE WITH GPT4O-MINI----------------------")
display(Markdown(f"{response_recursive_gpt4o_mini}"))

----------------------RESPONSE WITH O1 MINI----------------------


The replacement cost profit in oil production & operations URCP post taxation for the first quarter of 2020 was $392 million.

----------------------RESPONSE WITH O1 PREVIEW----------------------


The replacement cost profit in oil production and operations after taxation in the first quarter of 2020 is $392 million.

----------------------RESPONSE WITH GPT4O-MINI----------------------


The replacement cost profit (loss) in oil production & operations URCP post taxation in Q1 of 2020 is (2,798) million dollars.

## Observation:

Although for some queries `o1-mini` and `o1-preview` outperformed GPT-4o-mini, `o1-preview` encountered issues with Query-4.

Careful evaluation is necessary when considering the use of o1-models for Excel RAG.